<a href="https://colab.research.google.com/github/SheikhMudassarHanif/NLP/blob/main/trainingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##This is an extension of RnnTranslation file . I have preprocessed the data before in that file, in this i have imported that processed data and trainned an LSTM model with hpyerparameters vocabsize=15000 max_length=700 dim=50

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
#
# import tensorflow as tf
# from tensorflow import keras

In [3]:
#run this
excel_file_path = '/content/preprocessed_data.csv'
column_names = pd.read_csv(excel_file_path, nrows=0).columns.tolist()
df = pd.read_csv(excel_file_path,usecols=[column_names[0],column_names[1]])
df.dropna(inplace=True)
df.head()

,SENTENCES,MEANING
0,how can I commun with my parent,م اپ والدین سے کیسا با کرنا
1,how can I make friend,م دوس کیسا بنائ
2,whi do I get so sad,م اتنا اداس کم ہ
3,if youv ask yourself such question your not alon,اگر اپ اپ اپ سے ایسا سوالا کرنا ہ ت اپ اکیلے نہ ہ
4,depend on where youv turn for guidanc you may ...,میں با پرنا منحصر ہونا کہنا اپ رہنمائ کم لینا ...


In [4]:
#dont run
train,val,test=np.split(df.sample(frac=1),[int(0.8*len(df)),int(0.9*len(df))])

In [5]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [7]:
vocab_size=15000
embedding_dim=50
max_length=700
trunc_type='post'
oov_tok='<OOV>'
train_inputs=train['SENTENCES '].values
val_inputs=val['SENTENCES '].values
test_inputs=test['SENTENCES '].values


val_inputs_str=[str(x) for x in val_inputs]
test_inputs_str=[str(x) for x in test_inputs]
train_inputs_str = [str(x) for x in train_inputs]


#tokenization of input (english sentences)
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_inputs_str)

#vectorization of input

train_sequences = tokenizer.texts_to_sequences(train_inputs_str)
val_sequences = tokenizer.texts_to_sequences(val_inputs_str)
test_sequences = tokenizer.texts_to_sequences(test_inputs_str)

#Pad the sequences to ensure equal length
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)
val_padded = pad_sequences(val_sequences, maxlen=max_length, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)
train_labels=train['MEANING'].values
train_labels = [str(x) for x in train_labels]
tokenizer.fit_on_texts(train_labels)
train_labels_sequences = tokenizer.texts_to_sequences(train_labels)
padded_train_labels = pad_sequences(train_labels_sequences, maxlen=max_length, truncating=trunc_type)
val_labels=val['MEANING'].values
val_labels = [str(x) for x in val_labels]
# tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(val_labels)
val_labels_sequences = tokenizer.texts_to_sequences(val_labels)
padded_val_labels = pad_sequences(val_labels_sequences, maxlen=max_length, truncating=trunc_type)
test_labels=test['MEANING'].values
test_labels = [str(x) for x in test_labels]
# tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(test_labels)
test_labels_sequences = tokenizer.texts_to_sequences(test_labels)
padded_test_labels = pad_sequences(test_labels_sequences, maxlen=max_length, truncating=trunc_type)
# !pip install pickle
import pickle

# Save the tokenizer after fitting it on the training data
with open('sample_data/tokenizer15k800.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# model=tf.keras.models.load_model('/content/drive/MyDrive/best_model2.h5_64.keras')

In [ ]:



# model architure defined

model=tf.keras.Sequential([
    #this layer makes the input ready for nerual network
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=max_length),
    #LSTM layer
    tf.keras.layers.Dense(64,activation='relu'),

    tf.keras.layers.LSTM(64,return_sequences=True),
    #dropout layer
    tf.keras.layers.Dropout(0.3),
    #dense layer
tf.keras.layers.LSTM(64,return_sequences=True),
tf.keras.layers.LSTM(64,return_sequences=True),


    tf.keras.layers.Dense(64,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    #dense output layer using softmax
    tf.keras.layers.Dense(vocab_size,activation='softmax')



])

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

#defined callbacks to control the training flow epochs . such as if val_loss doesnt decrease stop . save the trained model and adjust the learning rate if needed
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),  # Stop early if val_loss doesn't improve
    tf.keras.callbacks.ModelCheckpoint(filepath='Lstmmodel15k800.keras', save_best_only=True, monitor='val_loss', verbose=1),  # Save the best model
  ]

# Fit the model with callbacks
historyRnn = model.fit(
    train_padded, padded_train_labels,
    batch_size=64, epochs=10,
    validation_data=(val_padded, padded_val_labels),
    callbacks=callbacks  # Add callbacks here
)



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.9468 - loss: 3.3911 
Epoch 1: val_loss improved from inf to 0.36657, saving model to Lstmmodel15k800.keras
370/370 ━━━━━━━━━━━━━━━━━━━━ 11353s 31s/step - accuracy: 0.9469 - loss: 3.3856 - val_accuracy: 0.9774 - val_loss: 0.3666
Epoch 2/10
370/370 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - accuracy: 0.9786 - loss: 0.2822 
Epoch 2: val_loss improved from 0.36657 to 0.25272, saving model to Lstmmodel15k800.keras
370/370 ━━━━━━━━━━━━━━━━━━━━ 11692s 32s/step - accuracy: 0.9786 - loss: 0.2821 - val_accuracy: 0.9782 - val_loss: 0.2527
Epoch 3/10
 13/370 ━━━━━━━━━━━━━━━━━━━━ 2:50:53 29s/step - accuracy: 0.9795 - loss: 0.1967

In [11]:
model.evaluate(test_padded,padded_test_labels)

93/93 ━━━━━━━━━━━━━━━━━━━━ 278s 3s/step - accuracy: 0.9716 - loss: 0.2632


[0.2720516324043274, 0.9705198407173157]

In [12]:
new_data = ["how can I commun with my parent"]
new_data = [str(x) for x in new_data]
tokenizer.fit_on_texts(new_data) # Update the tokenizer's vocabulary
new_sequences = tokenizer.texts_to_sequences(new_data) # Get the sequences
new_padded = tf.keras.preprocessing.sequence.pad_sequences(new_sequences, maxlen=max_length, truncating=trunc_type)
# Make predictions
predictions = model.predict(new_padded)
# View the prediction result
print(predictions)
predicted_sequence = np.argmax(predictions, axis=-1)
# Convert the predicted sequence of indices back to words
predicted_sentence = [tokenizer.index_word.get(index, '<OOV>') for index in predicted_sequence[0]]
# Join the words to form a sentence
predicted_sentence_str = ' '.join(predicted_sentence)
print(predicted_sentence_str)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[[9.7178704e-01 5.1757012e-04 4.7108650e-04 ... 2.4844252e-07
   6.8496213e-08 7.1557309e-08]
  [9.9343282e-01 1.5048731e-04 1.2959623e-04 ... 2.7707662e-08
   6.5372969e-09 6.8750876e-09]
  [9.9480224e-01 1.2270655e-04 1.0468037e-04 ... 1.9371688e-08
   4.4580424e-09 4.6936295e-09]
  ...
  [2.9389141e-02 2.9268993e-02 5.4404356e-02 ... 1.1820290e-05
   3.4899083e-06 3.4736777e-06]
  [2.8828152e-02 2.9433830e-02 5.4907750e-02 ... 1.1706482e-05
   3.4501779e-06 3.4331990e-06]
  [2.7273336e-02 2.9308474e-02 5.4824412e-02 ... 1.1963467e-05
   3.5449980e-06 3.5261294e-06]]]
<OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <O